In [ ]:
from pyspark import SparkContext, SparkConf
import pyspark.sql as sql
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType, IntegerType, ArrayType, StringType
from pyspark.sql.functions import udf, desc, col, max, sum, countDistinct, explode, rank
from typing import List
import os
import re

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!rm spark-3.5.5-bin-hadoop3.tgz
!wget -q http://archive.apache.org/dist/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz
!tar xf spark-3.5.5-bin-hadoop3.tgz

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.5-bin-hadoop3"

!pip install findspark # для нахождения spark в системе
import findspark
findspark.init()

In [ ]:
posts_path = 'posts_sample.xml'
langs_path = 'programming-languages.csv'

In [ ]:
spark = SparkSession \
  .builder \
  .appName("l2_2010_2020_programming_languages_data") \
  .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0")\
  .getOrCreate()

print("version spark:", spark.version)

version spark: 3.5.5


In [ ]:
posts_df = spark.read \
  .format('xml') \
  .options(rowTag='row') \
  .load(posts_path)
print("posts:")
posts_df.printSchema()

posts:
root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)



In [ ]:
# приведение списка языков к нижнему регистру, переименовывание в tag
langs_df = spark.read \
  .option("header", True) \
  .option("inferSchema", True) \
  .csv(langs_path) \
  .withColumn("tag", F.lower(col("name"))) \
  .select("tag")

print("programming languages:")
langs_df.printSchema()

programming languages:
root
 |-- tag: string (nullable = true)



In [ ]:
# udf для извлечения массива тегов
def extract_tags(tag_string):
  if not tag_string:
    return []
  return re.findall(r'<(.+?)>', tag_string)

# udf для извлечение года
def extract_year(ts):
  return ts.year

tags_udf = udf(extract_tags, ArrayType(StringType()))
year_udf = udf(extract_year, IntegerType())

# отбор колонок с тегами, годами и просмотрами
simplified_posts_df = posts_df \
  .withColumn("tags_list", tags_udf(col("_Tags"))) \
  .withColumn("year", year_udf(col("_LastActivityDate"))) \
  .select(col("tags_list"), col("year"), col("_ViewCount").alias("views"))

for idx, row in enumerate(simplified_posts_df.head(10), start=1):
  print(f"{idx}: {row}")

1: Row(tags_list=['c#', 'floating-point', 'type-conversion', 'double', 'decimal'], year=2019, views=42817)
2: Row(tags_list=['html', 'css', 'internet-explorer-7'], year=2019, views=18214)
3: Row(tags_list=[], year=2017, views=None)
4: Row(tags_list=['c#', '.net', 'datetime'], year=2019, views=555183)
5: Row(tags_list=['c#', 'datetime', 'time', 'datediff', 'relative-time-span'], year=2019, views=149445)
6: Row(tags_list=[], year=2018, views=None)
7: Row(tags_list=['html', 'browser', 'timezone', 'user-agent', 'timezone-offset'], year=2019, views=176405)
8: Row(tags_list=['.net', 'math'], year=2018, views=123231)
9: Row(tags_list=[], year=2010, views=None)
10: Row(tags_list=[], year=2010, views=None)


In [ ]:
# разбивка массива тегов в отдельные строки, приведение тега к нижнему регистру
exploded_tags_df = simplified_posts_df \
  .select(col("year"), explode(col("tags_list")).alias("tag_raw"), col("views")) \
  .withColumn("tag", F.lower(col("tag_raw"))) \
  .drop("tag_raw")

In [ ]:
# фильтрация данных, чтобы оставить записи с тегом в списке ЯП и годами 2010-2020
filtered_df = exploded_tags_df \
  .join(langs_df, on="tag", how="inner") \
  .filter((col("year") >= 2010) & (col("year") <= 2020))

In [ ]:
# группировка по году и ЯП, суммирование просмотров
agg_df = filtered_df \
  .groupBy("year", "tag") \
  .agg(sum("views").alias("total_views")) \
  .orderBy("year", desc("total_views"))

agg_df.show()

+----+------------+-----------+
|year|         tag|total_views|
+----+------------+-----------+
|2010|        java|      53333|
|2010|      matlab|      51865|
|2010| objective-c|      43878|
|2010|         php|      39730|
|2010|  javascript|      37059|
|2010|      python|      25930|
|2010|        ruby|      15864|
|2010|           c|      13810|
|2010|      delphi|       7680|
|2010|           r|       7499|
|2010|       xpath|       5122|
|2010|actionscript|       5104|
|2010|         sed|       4573|
|2010|        perl|       3515|
|2010|     haskell|       2601|
|2010|        bash|       2171|
|2010|       mouse|       1789|
|2010|      scheme|       1539|
|2010|      racket|       1539|
|2010| applescript|       1462|
+----+------------+-----------+
only showing top 20 rows



In [ ]:
# ранжирование внутри каждого года и отбор топ 10 по ранкам
window_by_year = Window.partitionBy("year").orderBy(desc("total_views"))
ranked_df = agg_df \
  .withColumn("rank", rank().over(window_by_year)) \
  .filter(col("rank") <= 10) \
  .select("year", "rank", "tag", "total_views") \
  .orderBy("year", "rank")

ranked_df.show(10, truncate=False)

+----+----+-----------+-----------+
|year|rank|tag        |total_views|
+----+----+-----------+-----------+
|2010|1   |java       |53333      |
|2010|2   |matlab     |51865      |
|2010|3   |objective-c|43878      |
|2010|4   |php        |39730      |
|2010|5   |javascript |37059      |
|2010|6   |python     |25930      |
|2010|7   |ruby       |15864      |
|2010|8   |c          |13810      |
|2010|9   |delphi     |7680       |
|2010|10  |r          |7499       |
+----+----+-----------+-----------+
only showing top 10 rows



In [ ]:
# сохранение отчета
output_path = "programming_languages_2010_2020_top10.parquet"
ranked_df.write \
  .partitionBy("year") \
  .mode("overwrite") \
  .parquet(output_path)

ranked_df.show()

+----+----+-----------+-----------+
|year|rank|        tag|total_views|
+----+----+-----------+-----------+
|2010|   1|       java|      53333|
|2010|   2|     matlab|      51865|
|2010|   3|objective-c|      43878|
|2010|   4|        php|      39730|
|2010|   5| javascript|      37059|
|2010|   6|     python|      25930|
|2010|   7|       ruby|      15864|
|2010|   8|          c|      13810|
|2010|   9|     delphi|       7680|
|2010|  10|          r|       7499|
|2011|   1|       java|     121315|
|2011|   2|     python|      89637|
|2011|   3|          c|      73116|
|2011|   4|        php|      67341|
|2011|   5| javascript|      61631|
|2011|   6|objective-c|      54815|
|2011|   7|          r|      14394|
|2011|   8|       ruby|       9771|
|2011|   9|     cython|       8109|
|2011|  10|     delphi|       6724|
+----+----+-----------+-----------+
only showing top 20 rows



In [ ]:
# удаление отчета
import shutil
shutil.rmtree(output_path)